In [1]:
import braket
import boto3
import numpy as np
import matplotlib.pyplot as plt
import json
import os.path

from braket.aws import AwsQuantumTask
from braket.tracking import Tracker
tracker = Tracker().start()

In [2]:
megahertz = 1e+6
micron = 1e-6
microsecond = 1e-6
NUM_ATOMS = 4

In [3]:
from ResQ_utils import pulseTimeSeries, get_counts, simulatedResidualInference, residualInferenceProgram

def tally_votes(sim_counts, numShots=10000):

    counter_0 = np.zeros(NUM_ATOMS)
    counter_1 = np.zeros(NUM_ATOMS)

    for k in counts.keys():
    
        for i, state in enumerate(k):
            if state == "d":
                counter_0[i] += counts[k]
            elif state == "u":
                counter_1[i] += counts[k]

    return np.sum(counter_1 / (numShots)) / NUM_ATOMS

def tally_votes_hardware(counts, numShots=1000):
    counter_0 = np.zeros(NUM_ATOMS)
    counter_1 = np.zeros(NUM_ATOMS)

    for k in counts.keys():
    
        for i, state in enumerate(k):
            if state == "g":
                counter_0[i] += counts[k]
            elif state == "r":
                counter_1[i] += counts[k]

    return np.sum(counter_1 / (numShots)) / NUM_ATOMS

In [4]:
classification_task = "trouserboot"

In [5]:
with open(f'input_data/{classification_task}datapoints.json') as f:
    data = np.array(json.load(f))

with open(f'input_data/{classification_task}datalabels.json') as f:
    test_labels = np.array(json.load(f))

with open(f'parameters/{classification_task}params.json') as f:
    d = json.load(f)

    opt_params = d["params"]
    latticeType = d["latticeType"]
    latticeConstant = d["latticeConstant"]

with open(f'run_details/{classification_task}run_details.json') as f:
    d = json.load(f)

    sampled_logits = d["simulated_logits"]
    indices =  d["data_index"]
    data_points = d["data_points"] 
    test_labels = d["test_labels"]

In [6]:
from braket.aws import AwsDevice
from braket.devices import Devices, LocalSimulator

device = AwsDevice(Devices.QuEra.Aquila)
sim_device = LocalSimulator("braket_ahs")

print(tracker.quantum_tasks_statistics())


{}


In [9]:
# double check once more
for i in range(8):
    d = data_points[i]
    
    program = residualInferenceProgram(d, opt_params, latticeType, latticeConstant)
    discretized_ahs_program = program.discretize(device)

    sim_result = sim_device.run(
       program,
       shots=10000,
        steps = 10000
    ).result() 
    counts = get_counts(sim_result)
    sim_discrete_logit = tally_votes(counts)

    print(sim_discrete_logit)
    

0.221575
0.23842499999999997
0.323275
0.47657499999999997
0.60885
0.74015
0.846125
0.8749


In [10]:
for l in sampled_logits: print(l)

0.22465
0.2436
0.32409999999999994
0.470275
0.6150249999999999
0.73695
0.8471249999999999
0.8772249999999999


In [ ]:
n_shots = 1000
task_arns = []

for i in range(8):
    print(i)
    d = data_points[i]
    
    program = residualInferenceProgram(d, opt_params, latticeType, latticeConstant)
    discretized_ahs_program = program.discretize(device)

    task = device.run(discretized_ahs_program, shots=n_shots)

    metadata = task.metadata()
    task_arn = metadata["quantumTaskArn"]
    task_status = metadata["status"]
    
    print(f"ARN: {task_arn}")
    print(f"status: {task_status}")

    task_arns.append(task_arn)
    

In [ ]:
task_arns

In [23]:
if not os.path.isfile(f'task_arns/{classification_task}task_arns.json'):
    with open(f'task_arns/{classification_task}task_arns.json', 'w') as f:
        json.dump(task_arns, f)
else:
    print("file exists, make sure you don't overwrite")

file exists, make sure you don't overwrite


In [45]:
for a in task_arns:
    task = AwsQuantumTask(arn=a)
    metadata = task.metadata()
    print(metadata["status"])

COMPLETED
COMPLETED
COMPLETED
COMPLETED
COMPLETED
COMPLETED
COMPLETED
COMPLETED


In [46]:
for a in task_arns:
    hardware_result = AwsQuantumTask(arn=a).result()
    hardware_counts = hardware_result.get_counts()
    print(tally_votes_hardware(hardware_counts, numShots=1000))

0.36024999999999996
0.39675
0.46950000000000003
0.5852499999999999
0.6172500000000001
0.46125000000000005
0.635
0.61375


In [73]:
for t in test_labels: print(t)

0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0


In [41]:
task = AwsQuantumTask(arn=task_arns[4])
metadata = task.metadata()
print(metadata["status"])

COMPLETED


In [42]:
result = task.result()
print(tally_votes_hardware(result.get_counts(), numShots=1000))

0.6172500000000001


In [47]:
result.get_counts()

{'rgrg': 29,
 'gggr': 46,
 'rggr': 51,
 'grrg': 35,
 'grgr': 37,
 'rgrr': 161,
 'rrrr': 85,
 'rggg': 33,
 'gggg': 45,
 'rrrg': 129,
 'rrgr': 110,
 'rrgg': 29,
 'grrr': 111,
 'errg': 2,
 'ggrg': 21,
 'ggrr': 32,
 'grgg': 27,
 'rreg': 1,
 'ggre': 1,
 'rrer': 3,
 'egrg': 1,
 'eggr': 1,
 'errr': 2,
 'egrr': 2,
 'rrre': 2,
 'regr': 1,
 'rerg': 1,
 'grre': 1,
 'rerr': 1}